https://www.kaggle.com/grantgasser/fraud-detection

In [3]:
for name in dir():
 if not name.startswith("_"):
   del globals()[name]

#Import necessary packages
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import pickle

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn import preprocessing
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, roc_curve, auc,precision_recall_curve

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
print(os.listdir("./data"))

input_path = './data'

#%matplotlib inline

RANDOM_SEED = 42
nan_replace = -999

['test_transaction.csv', '.DS_Store', 'ieee-fraud-detection-publicleaderboard.csv', 'train_identity.csv', 'pickles', 'results', 'test_identity.csv', 'random_out.csv', 'sample_submission.csv', 'train_transaction.csv']


In [6]:
train_transaction = pd.read_csv('./data/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('./data/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('./data/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('./data/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv('./data/sample_submission.csv', index_col='TransactionID')

In [7]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

print(train.shape)
print(test.shape)

(590540, 433)
(506691, 432)


In [8]:
y_train = train['isFraud'].copy()

In [9]:
# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [10]:
del train, test, train_transaction, train_identity, test_transaction, test_identity

In [11]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [16]:
clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=12,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999)

In [17]:
%%time
clf.fit(X_train, y_train)

CPU times: user 6h 17min 58s, sys: 1min 5s, total: 6h 19min 3s
Wall time: 33min 2s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=-999, n_estimators=500, n_jobs=12,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [18]:
sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
sample_submission.to_csv('./data/results/simple_xgboost.csv')